In [6]:
import sys
from pathlib import Path

# Dynamic project root (2 levels up if notebook is in notebooks/)
project_root = Path().resolve().parents[0]  # adjust depending on your notebook location
sys.path.append(str(project_root))

# Confirm path is added
print(sys.path)


['C:\\Program Files\\Python311\\python311.zip', 'C:\\Program Files\\Python311\\DLLs', 'C:\\Program Files\\Python311\\Lib', 'C:\\Program Files\\Python311', 'c:\\Users\\hamza\\OneDrive\\Desktop\\InterviewPrepUSA\\UCSC_Extension\\IntroToMachineLearning\\synthetic-finance-mlops\\venv', '', 'c:\\Users\\hamza\\OneDrive\\Desktop\\InterviewPrepUSA\\UCSC_Extension\\IntroToMachineLearning\\synthetic-finance-mlops\\venv\\Lib\\site-packages', 'c:\\Users\\hamza\\OneDrive\\Desktop\\InterviewPrepUSA\\UCSC_Extension\\IntroToMachineLearning\\synthetic-finance-mlops\\venv\\Lib\\site-packages\\win32', 'c:\\Users\\hamza\\OneDrive\\Desktop\\InterviewPrepUSA\\UCSC_Extension\\IntroToMachineLearning\\synthetic-finance-mlops\\venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\hamza\\OneDrive\\Desktop\\InterviewPrepUSA\\UCSC_Extension\\IntroToMachineLearning\\synthetic-finance-mlops\\venv\\Lib\\site-packages\\Pythonwin', 'C:\\Users\\hamza\\OneDrive\\Desktop\\InterviewPrepUSA\\UCSC_Extension\\IntroToMachineLearn

In [7]:
from src.train import run_anomaly_supervised_pipeline
from src.evaluate import evaluate_models


In [8]:
path=r'C:\Users\hamza\OneDrive\Desktop\InterviewPrepUSA\UCSC_Extension\IntroToMachineLearning\synthetic-finance-mlops\data\raw'
models,xt,yt=run_anomaly_supervised_pipeline(path)



🔧 Computing transaction features...
  → Amount-based features
  → Account status features
  → Customer demographic features
  → Loan-related features
  → Temporal features
  → Anomaly detection flags
✅ Feature engineering completed!
✅ Preprocessing complete. Data ready for model prep.
✅ Model data prepared (scaled).

🔹 Training Logistic Regression...


c:\Users\hamza\OneDrive\Desktop\InterviewPrepUSA\UCSC_Extension\IntroToMachineLearning\synthetic-finance-mlops\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 5000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=5000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



🔹 Training Random Forest...

🔹 Training XGBoost...


c:\Users\hamza\OneDrive\Desktop\InterviewPrepUSA\UCSC_Extension\IntroToMachineLearning\synthetic-finance-mlops\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:55:23] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [9]:
results=evaluate_models(models,xt,yt)

In [10]:
results.head()

,ROC-AUC,PR-AUC,Precision,Recall,Accuracy,Balanced Accuracy,Confusion Matrix,Classification Report
model,,,,,,,,
Logistic Regression,0.830407,0.327840,0.102110,0.720238,0.7778,0.750020,"[[7536, 2128], [94, 242]]",precision recall f1-score ...
Random Forest,0.984710,0.754946,0.889655,0.383929,0.9777,0.691136,"[[9648, 16], [207, 129]]",precision recall f1-score ...
XGBoost,0.986007,0.771929,0.598291,0.833333,0.9756,0.906940,"[[9476, 188], [56, 280]]",precision recall f1-score ...


In [11]:
import requests
import pandas as pd

# Load your test dataset
X_test = xt  # adjust path if needed

# Convert to JSON format expected by MLflow serve
payload = {
    "columns": list(X_test.columns),
    "data": X_test.values.tolist()
}

# MLflow model server URL
url = "http://127.0.0.1:1234/invocations"

# Send POST request
response = requests.post(url, json=payload)

# Check status
if response.status_code == 200:
    predictions = response.json()
    print("✅ Predictions received:")
    print(predictions)
else:
    print(f"❌ Request failed with status {response.status_code}: {response.text}")


❌ Request failed with status 400: {"error_code": "BAD_REQUEST", "message": "The input must be a JSON dictionary with exactly one of the input fields {'dataframe_split', 'instances', 'inputs', 'dataframe_records'}. Received dictionary with input fields: []. IMPORTANT: The MLflow Model scoring protocol has changed in MLflow version 2.0. If you are seeing this error, you are likely using an outdated scoring request format. To resolve the error, either update your request format or adjust your MLflow Model's requirements file to specify an older version of MLflow (for example, change the 'mlflow' requirement specifier to 'mlflow==1.30.0'). If you are making a request using the MLflow client (e.g. via `mlflow.pyfunc.spark_udf()`), upgrade your MLflow client to a version >= 2.0 in order to use the new request format. For more information about the updated MLflow Model scoring protocol in MLflow 2.0, see https://mlflow.org/docs/latest/models.html#deploy-mlflow-models."}


In [13]:
import requests
import pandas as pd

# X_test is your DataFrame
payload = {
    "dataframe_split": {
        "columns": list(xt.columns),
        "data": xt.values.tolist()
    }
}

url = "http://127.0.0.1:1234/invocations"

response = requests.post(url, json=payload)

if response.status_code == 200:
    predictions = response.json()
    print("✅ Predictions received:")
    print(predictions)
else:
    print(f"❌ Request failed with status {response.status_code}: {response.text}")


✅ Predictions received:
{'predictions': [0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,

In [16]:
yt.values.tolist()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
